In [35]:
import argparse
from copy import copy
import inspect
import os.path
import sys

import ipywidgets as widgets

from symdesign import flags
from symdesign.SymDesign import app

In [39]:
def get_class_that_defined_method(meth):
    """Where meth is a bound method"""
    if inspect.ismethod(meth):
        for cls in inspect.getmro(meth.__self__.__class__):
            if meth.__name__ in cls.__dict__:
                return cls
    return None


# All modules
# flags.available_modules
notebook_allowed_modules = [
    flags.align_helices,
    flags.analysis,
    flags.check_clashes,
    flags.cluster_poses,
    flags.design,
    flags.expand_asu,
    flags.generate_fragments,
    flags.helix_bending,
    flags.protocol,
    flags.rename_chains,
    flags.select_designs,
    flags.select_poses,
    flags.select_designs,
]
module_help = widgets.Label(value='Please input the module(s) you would like to include with this job')
module_tags = widgets.TagsInput(
    value=[flags.align_helices],
    allowed_tags=notebook_allowed_modules,
    allow_duplicates=False
)
module_input = widgets.VBox([module_help, module_tags])
display(module_input)

In [67]:
choices_widget = widgets.Dropdown
multiple_text = widgets.TagsInput
single_text = widgets.Text

def process_module_arg_to_widget(arg, widget_kwargs):
    if arg.nargs:
        widget = multiple_text(**widget_kwargs)
    else:
        if arg.metavar:
            placeholder = arg.metavar
        else:
            placeholder = ''  # arg.default
        widget = single_text(**widget_kwargs, placeholder=placeholder)

    return widget


none_tuple = (None,)
# Add each requested as its own widget, formatting module arguments
all_module_widgets = []  # widgets.IntSlider(), widgets.Text()]
# for group in parser._action_groups:
for group in flags.argparsers[flags.parser_entire]._action_groups:
    for arg in group._group_actions:
        if isinstance(arg, argparse._SubParsersAction):
            for module, subparser in arg.choices.items():
                if module in module_tags.value:
                    module_description = widgets.Label(value=subparser.description)
                    module_widgets = []
                    for group in subparser._action_groups:
                        # These are the processed module arguments
                        for module_arg in group._group_actions:
                            # description = f'--{module_arg.dest}'
                            # print(module_arg)
                            # help = module_arg.help
                            choices = module_arg.choices
                            required = module_arg.required
                            type_ = module_arg.type
                            
                            if module_arg.default is None:
                                if module_arg.nargs:
                                    module_value = []
                                    # choices = none_tuple + copy(choices)
                                else:
                                #     format_widget = lambda widget: widget
                                    module_value = module_arg.default
                            elif isinstance(module_arg.default, bool):
                                module_value = module_arg.default
                            elif isinstance(module_arg.default, (int, float)):
                                module_value = str(module_arg.default)
                            else:
                                module_value = module_arg.default
                                
                            # try:
                            widget_kwargs = dict(value=module_value,
                                                 description=module_arg.option_strings[-1],
                                                 tooltip=module_arg.help % vars(module_arg))
                            # except TypeError as error:
                                # print(error)
                                # input(module_arg.help)
                            if isinstance(module_arg, (argparse._StoreTrueAction, argparse._StoreFalseAction,
                                                       argparse.BooleanOptionalAction)):  # .action in boolean_actions:
                                if isinstance(module_arg, argparse.BooleanOptionalAction):
                                    # Swap the "--no-" prefixed flag for the typical prefix
                                    widget_kwargs.pop('description')
                                    widget_kwargs['description'] = module_arg.option_strings[-2]
                                widget = widgets.ToggleButton(**widget_kwargs)  # , tooltip=help)
                            elif isinstance(module_arg, argparse._StoreAction):
                                if choices:
                                    if module_value not in choices:
                                        # When there is no default provided
                                        choices_ = (module_value, *choices)
                                    else:
                                        choices_ = choices
                                    widget = choices_widget(**widget_kwargs, options=choices_)
                                elif type_ is None or get_class_that_defined_method(type_) == str:
                                    # These are processed as strings
                                    widget = process_module_arg_to_widget(module_arg, widget_kwargs)
                                    # if module_arg.nargs:
                                    #     widget = multiple_text(**widget_kwargs)
                                    # else:
                                    #     if module_arg.metavar:
                                    #         placeholder = module_arg.metavar
                                    #     else:
                                    #         placeholder = ''
                                    #     widget = single_text(**widget_kwargs, placeholder=placeholder)
                                elif type_ == int:
                                    widget = process_module_arg_to_widget(module_arg, widget_kwargs)
                                    # if module_arg.nargs:
                                    #     # There are currently none of these
                                    #     widget = widgets.IntsInput(**widget_kwargs)
                                    # else:
                                    #     widget = widgets.IntText(**widget_kwargs)
                                elif type_ == float:
                                    widget = process_module_arg_to_widget(module_arg, widget_kwargs)
                                    # if module_arg.nargs:
                                    #     # There are currently none of these
                                    #     widget = widgets.FloatsInput(**widget_kwargs)
                                    # else:
                                    #     widget = widgets.FloatText(**widget_kwargs)
                                    # widget = widgets.HBox([widget, widgets.Label(value=module_arg.metavar)])
                                elif type_ == os.path.abspath:
                                    widget = process_module_arg_to_widget(module_arg, widget_kwargs)
                                    # if module_arg.nargs:
                                    #     # There are currently none of these
                                    #     widget = multiple_text(**widget_kwargs, placeholder=module_arg.metavar)
                                    # else:
                                    #     if module_arg.metavar:
                                    #         placeholder = module_arg.metavar
                                    #     else:
                                    #         placeholder = ''
                                    #     widget = single_text(**widget_kwargs, placeholder=placeholder)
                                # Custom types
                                elif type_ == flags.temp_gt0:
                                    widget = process_module_arg_to_widget(module_arg, widget_kwargs)
                                    # widget = widgets.FloatsInput(**widget_kwargs)
                            else:
                                continue
                            # widgets.HBox([widget, widgets.Label(value=module_arg.help)])
                            # Add the module widgets to all widgets
                            module_widgets.append(widget)  # format_widget(widget))
                    
                    max_description_length = 0
                    for widget in module_widgets:
                        if len(widget.description) > max_description_length:
                            max_description_length = len(widget.description)
                            print(max_description_length)
                            print(widget.style)

                    module_layout = {'description_width': f'{max_description_length * 6}px'}
                    print(max_description_length * 5)
                    for widget in module_widgets:
                        widget.style = module_layout
                    # Add the module widgets to all widgets
                    all_module_widgets.append(widgets.VBox(module_widgets))
# For formatting the help, use
# widgets.HBox([WIDGET, widgets.Label(value="The $m$ in $E=mc^2$:")])
# for boolean flags, use
# widgets.ToggleButton(
#     value=False,
#     description='Click me',
#     disabled=False,
#     button_style='', # 'success', 'info', 'warning', 'danger' or ''
#     tooltip='Description',
#     icon='check' # (FontAwesome names without the `fa-` prefix)
# )
# For choices flags, use
# widgets.Dropdown(
#     options=[('One', 1), ('Two', 2), ('Three', 3)],
#     # options=['1', '2', '3'],
#     value=2,
#     description='Number:',
# )
# For int flags, use
# widgets.IntText(
#     value=7,
#     description='Any:',
#     disabled=False
# )
# widgets.IntsInput(
#     value=[1, 4, 3243],
#     min=0,
#     max=1000000,
#     format='$,d'
# )
# For float flags, use
# widgets.FloatText(
#     value=7,
#     description='Any:',
#     disabled=False
# )
# For string flags, use
# widgets.Text(
#     value='Hello World',
#     placeholder='Type something',
#     description='String:',
#     disabled=False
# )

# Set up the display to show all options for each module
# # Accordian display
# options_accordion = widgets.Accordion(children=all_module_widgets, titles=module_tags.value)
# display(options_accordion)
# Tab display
options_tab = widgets.Tab()
options_tab.children = all_module_widgets
options_tab.titles = module_tags.value  # ['align', 'design']
display(options_tab)

15
TextStyle()
16
DescriptionStyle()
80
11
ToggleButtonStyle()
16
ToggleButtonStyle()
23
ToggleButtonStyle()
115


In [1]:
# Format the arguments input from user into parsable command-line like arguments
if len(module_tags.value) > 1:
    sys.argv = ['protocol']
else:
    sys.argv = ['protocol', '--module'] + module_tags.value

In [38]:
run_protocol_button = widgets.Button(
    description='Run protocol',
    disabled=False,
    button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='To perform the protocols specified, click this button',
    icon='turn-down-left'
)
run_protocol_button.on_click(app)
display(run_protocol_button)

Button(button_style='success', description='Run protocol', icon='turn-down-left', style=ButtonStyle(), tooltip…

In [31]:
from collections import defaultdict
from IPython.display import HTML
import ipywidgets
from pprint import pprint

In [32]:
reverse_lut = defaultdict(set)
styles = set()
for export_name in dir(ipywidgets.widgets):
    export = getattr(ipywidgets.widgets, export_name)
    try:
        if issubclass(export, ipywidgets.Widget) and 'style' in export.class_trait_names():
            reverse_lut[export.style.klass.__name__].add(export.__name__)
            styles.add(export.style.klass)
    except TypeError:
        pass

html = '<ul>'
for style, widgets_ in reverse_lut.items():
    html = f"{html}\n<li><b>{style}:</b> {', '.join(sorted(widgets_))}</li>"
html += "</ul>"
HTML(html)

In [62]:
attributes = defaultdict(set)
base_traits = set(ipywidgets.Style.class_trait_names())

for s in styles:
    for t in s.class_trait_names():
        if not t.startswith("_") and t not in base_traits:
            attributes[s.__name__].add(t)
all_attributes = set().union(*attributes.values())

html = '<table>\n'
html = f"{html}<tr><th>Attribute</th>{ ''.join(f'<th>{s}</th>' for s in attributes.keys()) }</tr>"
for a in all_attributes:
    html = f"""{html}<tr><td>{a}</td>{ ''.join(f'<td>{"✓" if a in attribs else ""}</td>' for attribs in attributes.values()) }</tr>"""
html += "</table>"
HTML(html)

Attribute,LabelStyle,SliderStyle,CheckboxStyle,DescriptionStyle,HTMLStyle,ToggleButtonStyle,ToggleButtonsStyle,TextStyle,HTMLMathStyle,ProgressStyle,ButtonStyle
background,✓,,✓,,✓,,,✓,✓,,
font_weight,✓,,,,,✓,✓,,,,✓
handle_color,,✓,,,,,,,,,
description_width,✓,✓,✓,✓,✓,✓,✓,✓,✓,✓,
font_variant,✓,,,,,✓,,,,,✓
button_width,,,,,,,✓,,,,
text_decoration,✓,,,,,✓,,,,,✓
font_family,✓,,,,,✓,,,,,✓
font_size,✓,,,,✓,✓,,✓,✓,,✓
font_style,✓,,,,,✓,,,,,✓
